In [ ]:
from bioneuralnet.datasets import load_parkinsons_data

# Load PD transcriptomics data (GSE165082)
expression_df, sample_meta, gene_meta = load_parkinsons_data()

expression_df.head(), sample_meta['condition'].value_counts(), gene_meta.head()
